# 从ratings_Sports_and_Outdoors.csv文件中提取U-I交互图, 5-core后重新编号
- Extracting U-I interactions and performing 5-core, re-indexing
- dataset located at: http://jmcauley.ucsd.edu/data/amazon/links.html, rating only file in "Small" subsets for experimentation

In [47]:
import os, csv
import pandas as pd
import json
import pickle
import numpy as np

In [48]:
os.chdir('/home/yueqi/peilin/MMRec/AmazonRawData/Amazon_Review_Plus')
os.getcwd()

dataset = 'toys'
datamapsPath = f'./{dataset}/datamaps.json'
item2sidePath = f'./{dataset}/item2side.json'
negativeSamplesPath  = f'./{dataset}/negative_samples.txt'
purchaseHistoryPath  = f'./{dataset}/purchase_history.pkl'
rslt_dir = f'/home/yueqi/peilin/MMRec/data/AmazonReview/{dataset}/'
if not os.path.exists(rslt_dir):
    os.makedirs(rslt_dir)

## 先5-core过滤
## 5-core filtering

In [49]:

with open(purchaseHistoryPath, 'rb') as purchasefile:
    purchaseHistorys = pickle.load(purchasefile)

userID = 0
interactionData = {
    'userID': [],
    'itemID': [],
    'rating': [],
    'timestamp': [],
    'x_label': []
}

for purchaseHistory in purchaseHistorys:
    userID += 1
    for index, itemID in enumerate(purchaseHistory):
        interactionData['userID'].append(int(userID)-1)
        interactionData['itemID'].append(int(itemID)-1)
        # we don't care about rating & timestamp
        interactionData['rating'].append(0.0)
        interactionData['timestamp'].append(0)
        if index == len(purchaseHistory) - 2:
            interactionData['x_label'].append(1)
        elif index == len(purchaseHistory) - 1:
            interactionData['x_label'].append(2)
        else:
            interactionData['x_label'].append(0)

interactionDataFrame = pd.DataFrame(interactionData)
print(interactionDataFrame.head())
    
    
    
    
# df = pd.read_csv('ratings_Sports_and_Outdoors.csv', names=['userID', 'itemID', 'rating', 'timestamp'], header=None)
# print(f'shape: {df.shape}')
# df[:5]


   userID  itemID  rating  timestamp  x_label
0       0       0     0.0          0        0
1       0       1     0.0          0        0
2       0       2     0.0          0        0
3       0       3     0.0          0        0
4       0       4     0.0          0        0


In [50]:
with open(datamapsPath, 'r', encoding='utf-8') as datamapsFile:
    datamaps = json.load(datamapsFile)

count = 0
for items in datamaps.items():     
    if count < 10:
        print(items)
    count += 1
    
u_id_map = datamaps['user2id']
i_id_map = datamaps['item2id']



('user2id', {'AQW172A5SG5OC': '1', 'A3E9UK6ANIX73Q': '2', 'A38N7SFDOB3MR3': '3', 'A1BR7B2D7QXL83': '4', 'A3JLBL62FPGY98': '5', 'A3VNB1W7N079T2': '6', 'A1P9UMP1XSE6MI': '7', 'A18AO94P2ZIEF3': '8', 'A2BRAVOAYRHDVR': '9', 'A2F0AYUD1X0AMV': '10', 'A2B8O4VOUC1Y5B': '11', 'A1LG1DKUZXGJ6J': '12', 'A18CF71GKLDS48': '13', 'A2BFEMPHZAZE80': '14', 'A2UAQ83AY3C3CY': '15', 'A7EJERNB7IK3Z': '16', 'A3MKKQT9ECS0S6': '17', 'A279XQT50MB44R': '18', 'A2B0KZ54I63XNX': '19', 'A1D8KV88A3D7IZ': '20', 'A1CYKFUPE5ZGBA': '21', 'A2X6FC7OXFB61U': '22', 'A37XBQWYQ1N1FF': '23', 'A3FGWQHVQGOAGL': '24', 'A17LDNW8GIFID1': '25', 'AK9N3O4FT8DNQ': '26', 'A198M311Q3VRYF': '27', 'AN05GBGE65X9J': '28', 'A35NGY49ZY896M': '29', 'A1BP92HYW0HQST': '30', 'A144U5041CFKQR': '31', 'A3VAGT9TQUS9WE': '32', 'A2RXOQQCU6Z2X8': '33', 'A2OETFZ7IA6XSQ': '34', 'A1N03V52IRZ5PR': '35', 'A17B2QIN8RVRBJ': '36', 'AO9XKAUG5XH22': '37', 'A3G7BEJJCPD6DS': '38', 'A1V23RXLRBR73T': '39', 'A3N9DRV2RSEU6Y': '40', 'A25OJH02IZ86EP': '41', 'A2OD23539QMZS5':

In [51]:
# from collections import Counter


# min_u_num, min_i_num = 5, 5

# def get_illegal_ids_by_inter_num(df, field, max_num=None, min_num=None):
#     if field is None:
#         return set()
#     if max_num is None and min_num is None:
#         return set()

#     max_num = max_num or np.inf
#     min_num = min_num or -1

#     ids = df[field].values
#     inter_num = Counter(ids)
#     ids = {id_ for id_ in inter_num if inter_num[id_] < min_num or inter_num[id_] > max_num}
#     print(f'{len(ids)} illegal_ids_by_inter_num, field={field}')

#     return ids


# def filter_by_k_core(df):
#     while True:
#         ban_users = get_illegal_ids_by_inter_num(df, field=learner_id, max_num=None, min_num=min_u_num)
#         ban_items = get_illegal_ids_by_inter_num(df, field=course_id, max_num=None, min_num=min_i_num)
#         if len(ban_users) == 0 and len(ban_items) == 0:
#             return

#         dropped_inter = pd.Series(False, index=df.index)
#         if learner_id:
#             dropped_inter |= df[learner_id].isin(ban_users)
#         if course_id:
#             dropped_inter |= df[course_id].isin(ban_items)
#         print(f'{len(dropped_inter)} dropped interactions')
#         df.drop(df.index[dropped_inter], inplace=True)



## k-core

In [52]:
# filter_by_k_core(df)
# print(f'k-core shape: {df.shape}')
# print(f'shape after k-core: {df.shape}')
# df[:2]

## Re-index

In [53]:
# df.reset_index(drop=True, inplace=True)

In [54]:

i_mapping_file = 'i_id_mapping.csv'
u_mapping_file = 'u_id_mapping.csv'

# dump

u_df = pd.DataFrame(list(u_id_map.items()), columns=['user_id', 'userID'])
i_df = pd.DataFrame(list(i_id_map.items()), columns=['asin', 'itemID'])
u_df['userID'] = u_df['userID'].apply(int) - 1
i_df['itemID'] = i_df['itemID'].apply(int) - 1

u_df.to_csv(os.path.join(rslt_dir, u_mapping_file), sep='\t', index=False)
i_df.to_csv(os.path.join(rslt_dir, i_mapping_file), sep='\t', index=False)
print(f'mapping dumped...')

mapping dumped...


In [55]:
rslt_file = f'{dataset}.inter'

interactionDataFrame.to_csv(os.path.join(rslt_dir, rslt_file), sep='\t', index=False)
interactionDataFrame[:5]
#print('done!')

,userID,itemID,rating,timestamp,x_label
0,0,0,0.0,0,0
1,0,1,0.0,0,0
2,0,2,0.0,0,0
3,0,3,0.0,0,0
4,0,4,0.0,0,0


## Reload

In [56]:
indexed_df = pd.read_csv(os.path.join(rslt_dir, rslt_file), sep='\t')
print(f'shape: {indexed_df.shape}')
indexed_df[:4]

shape: (167597, 5)


,userID,itemID,rating,timestamp,x_label
0,0,0,0.0,0,0
1,0,1,0.0,0,0
2,0,2,0.0,0,0
3,0,3,0.0,0,0


In [57]:
u_uni = indexed_df['userID'].unique()
c_uni = indexed_df['itemID'].unique()

print(f'# of unique learners: {len(u_uni)}')
print(f'# of unique courses: {len(c_uni)}')

print('min/max of unique learners: {0}/{1}'.format(min(u_uni), max(u_uni)))
print('min/max of unique courses: {0}/{1}'.format(min(c_uni), max(c_uni)))


# of unique learners: 19412
# of unique courses: 11924
min/max of unique learners: 0/19411
min/max of unique courses: 0/11923
